# Hopfield Newtorks

###### Simple implementation of a Hopfield network used to restore images

TODO: Add more images, try grayscale? (map things to a line or sigmoid instead of +/-1)  
Note: The bias term is always assumed to be zero here. There is however no loss in generality since it is not a learnable parameter and acts as a constant effect in the total network energy. 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from os import listdir
from os.path import isfile, join

In [ ]:
def base_patterns(x=64, y=64):
    res = []
    onlyfiles = [join('input_pics', f) for f in listdir('input_pics') if isfile(join('input_pics', f))]
    n = len(onlyfiles)
    fig, ax = plt.subplots(1, n, figsize=(18,6))
    try:
        axes = ax.ravel()
    except: # when n == 1
        axes = []
        axes.append(ax)
    i = 0
    for f in onlyfiles:
        # Load image, cut down to one channel, convert values to -1 or 1
        im = np.array(Image.open(f)) - 127.5
        if len(im.shape) > 2:
            im = im[:,:,0]
        im = im[:x, :y]  # Crop as needed
        im[im < 0] = -1
        im[im > 0] = 1
        im = -im
        axes[i].imshow(im, cmap='binary')
        axes[i].set_title('pattern {}'.format(i))
        im = np.reshape(im, (x * y))
        res.append(im)
        i += 1
    return np.array(res).T

def energy(state, W):
    return -0.5 * state @ W @ state  # @ symbol is matrix multiply

width = 128
height = 128
X = base_patterns(width,height)

In [ ]:
# we build the weight matrix like this, c being the "learning rate"
c = 1
M = c * (1 / (width * height)) * (X @ X.T)
# We also need to zero the diagonal
np.fill_diagonal(M, 0)
plt.imshow(M)
plt.colorbar()

In [ ]:
for i in range(len(X[0])):
    # Randomly pick a corrupted pattern to recover
    np.random.seed()
    which = np.random.randint(0, len(X[0]))
    which = i
    # how corrupted 
    sigma = 1
    vt = sigma * np.random.randn((width * height))# + X[:,which]
    vt[vt > 0] = 1
    vt[vt < 0] =-1

    plt.imshow(np.reshape(vt, (width, height)), cmap='binary')
    title = 'We will try to recover pattern {} \n Corrupted with sigma = {} gaussian noise'.format(which, sigma)
    _ = plt.title(title)

    # save the evolution
    ev = []
    en = []
    energy_old = np.infty
    energy_new = energy(vt, M)
    steps = 200
    iteration = 0
    # we keep running until we reach the lowest energy level
    while (energy_old > energy_new) and iteration < steps: 
        iteration += 1
        energy_old = energy_new
        ev.append(np.copy(vt))
        en.append(energy_old)
        # -------------------------------------------------
        # This is asynchronous update v1
        # for ind in np.random.permutation(range(len(vt))):
        #    vt[ind] = np.sign(M[ind,:] @ vt) 
        # -------------------------------------------------
        # This is asynchronous update v2
        for pixel in np.split(np.random.randint(0, len(vt), width * height), 8):
            vt[pixel] = np.sign(M[pixel,:] @ vt)
        # -------------------------------------------------
        # This is synchronous update
        #vt = np.sign(M @ vt)
        # -------------------------------------------------
        energy_new = energy(vt, M)
    print('Stopped at iteration {}'.format(iteration))

    fig, ax = plt.subplots(1,len(ev), figsize=(18,6))
    axes = ax.ravel()
    fig.suptitle('Trying to reconstruct pattern {} lowering network energy'.format(which))
    for idx in range(len(ev)):
        axes[idx].imshow(np.reshape(ev[idx], (width,height)),cmap='binary')
        axes[idx].set_title('Iteration {} \n Net. energy: {:.2f}'.format(idx, en[idx]))

In [ ]:
# energy decreases monotonically at every iteration
plt.plot(en)